# youtube-dl here

In [1]:
!pip -q install lxml youtube-dl
#!wget -c https://youtube-dl.org/downloads/latest/youtube-dl-2020.12.26.tar.gz -O - | tar -xz

In [ ]:
# For google colab notebooks
from google.colab import drive
drive.mount('/content/drive')

In [13]:
import youtube_dl

ydl = youtube_dl.YoutubeDL({'outtmpl': '%(id)s.%(ext)s'})

with ydl:
    result = ydl.extract_info(
        'https://www.youtube.com/playlist?list=PLv-hwRQ7MOquDiJCO3iZsrTS4_QiGQ6gz',
        download=False # We just want to extract the info
    )

if 'entries' in result:
    # Can be a playlist or a list of videos
    video = result['entries'][0]
else:
    # Just a video
    video = result

[youtube:tab] PLv-hwRQ7MOquDiJCO3iZsrTS4_QiGQ6gz: Downloading webpage
[download] Downloading playlist: Classic Fairy Tales
[youtube:tab] playlist Classic Fairy Tales: Downloading 15 videos
[download] Downloading video 1 of 15
[youtube] BXDsucz23OA: Downloading webpage
[download] Downloading video 2 of 15
[youtube] YoQyyB5xvLk: Downloading webpage
[download] Downloading video 3 of 15
[youtube] 3QzT1sq6kCY: Downloading webpage
[download] Downloading video 4 of 15
[youtube] Xd97k149tvo: Downloading webpage
[download] Downloading video 5 of 15
[youtube] qOJ_A5tgBKM: Downloading webpage
[download] Downloading video 6 of 15
[youtube] gzZbtMFP-_I: Downloading webpage
[download] Downloading video 7 of 15
[youtube] LDMWJCrDVMI: Downloading webpage
[download] Downloading video 8 of 15
[youtube] X4o5_8cEAHs: Downloading webpage
[download] Downloading video 9 of 15
[youtube] -QdRcUeKFuQ: Downloading webpage
[download] Downloading video 10 of 15
[youtube] -QdRcUeKFuQ: Downloading webpage
[download]

In [16]:
## Avaliable keys for template use
print('\n'.join(video.keys()))

id
uploader
uploader_id
uploader_url
channel_id
channel_url
upload_date
license
creator
title
alt_title
thumbnails
description
categories
tags
subtitles
automatic_captions
duration
age_limit
annotations
chapters
webpage_url
view_count
like_count
dislike_count
average_rating
formats
is_live
start_time
end_time
series
season_number
episode_number
track
artist
album
release_date
release_year
extractor
webpage_url_basename
extractor_key
n_entries
playlist
playlist_id
playlist_title
playlist_uploader
playlist_uploader_id
playlist_index
thumbnail
display_id
requested_subtitles
requested_formats
format
format_id
width
height
resolution
fps
vcodec
vbr
stretched_ratio
acodec
abr
ext


In [7]:
#ydl_opts = {
#    'format': 'bestaudio/best',
#    'download_archive': 'downloaded_songs.txt',
#    'outtmpl': '%(title)s.%(ext)s', #/content/drive/MyDrive/Fast.io/learnportal.imfast.io/uploads/%(uploader)_
#    'postprocessors': [{
#        'key': 'FFmpegExtractAudio',
#        'preferredcodec': 'mp3',
#        'preferredquality': '192',
#        }]
#}

ydl_opts = {
    'outtmpl': '%(upload_date)s_%(title)s.%(ext)s', #/content/drive/MyDrive/Fast.io/learnportal.imfast.io/uploads/%(uploader)s/
    'format': 'best',
    'postprocessors': [{
        'key': 'FFmpegVideoConvertor',
        'preferedformat': 'mp4'
        }]    
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/playlist?list=PLv-hwRQ7MOquDiJCO3iZsrTS4_QiGQ6gz'])

[youtube:tab] PLD15D38DC7AA3B737: Downloading webpage
[download] Downloading playlist: Economics 421 - Econometrics - Winter 2011
[youtube:tab] playlist Economics 421 - Econometrics - Winter 2011: Downloading 19 videos
[download] Downloading video 1 of 19
[youtube] WK03XgoVsPM: Downloading webpage
[download] Destination: 20110106_Economics 421_521 - Econometrics - Winter 2011 - Lecture 1 (HD).f137.mp4
[download] 100% of 676.22MiB in 00:2591MiB/s ETA 00:00known ETA
[download] Destination: 20110106_Economics 421_521 - Econometrics - Winter 2011 - Lecture 1 (HD).f140.m4a
[download] 100% of 71.23MiB in 00:0497MiB/s ETA 00:0052
[ffmpeg] Merging formats into "20110106_Economics 421_521 - Econometrics - Winter 2011 - Lecture 1 (HD).mp4"
Deleting original file 20110106_Economics 421_521 - Econometrics - Winter 2011 - Lecture 1 (HD).f137.mp4 (pass -k to keep)
Deleting original file 20110106_Economics 421_521 - Econometrics - Winter 2011 - Lecture 1 (HD).f140.m4a (pass -k to keep)
[download] Dow

[download] Destination: 20110211_Economics 421_521 - Econometrics - Winter 2011 - Lecture 11 (HD).f137.mp4
[download] 100% of 1.41GiB in 01:4089MiB/s ETA 00:0036own ETA
[download] Destination: 20110211_Economics 421_521 - Econometrics - Winter 2011 - Lecture 11 (HD).f251.webm
[download] 100% of 66.19MiB in 00:0502MiB/s ETA 00:008
[ffmpeg] Merging formats into "20110211_Economics 421_521 - Econometrics - Winter 2011 - Lecture 11 (HD).mkv"
Deleting original file 20110211_Economics 421_521 - Econometrics - Winter 2011 - Lecture 11 (HD).f137.mp4 (pass -k to keep)
Deleting original file 20110211_Economics 421_521 - Econometrics - Winter 2011 - Lecture 11 (HD).f251.webm (pass -k to keep)
[download] Downloading video 12 of 19
[youtube] Kb4LvSguwjg: Downloading webpage


[download] Destination: 20110216_Economics 421_521 - Econometrics - Winter 2011 - Lecture 12 (HD).f137.mp4
[download] 100% of 1.18GiB in 02:5978MiB/s ETA 00:005nown ETA
[download] Destination: 20110216_Economics 421_521 - Econometrics - Winter 2011 - Lecture 12 (HD).f251.webm
[download] 100% of 65.51MiB in 00:0665MiB/s ETA 00:001
[ffmpeg] Merging formats into "20110216_Economics 421_521 - Econometrics - Winter 2011 - Lecture 12 (HD).mkv"
Deleting original file 20110216_Economics 421_521 - Econometrics - Winter 2011 - Lecture 12 (HD).f137.mp4 (pass -k to keep)
Deleting original file 20110216_Economics 421_521 - Econometrics - Winter 2011 - Lecture 12 (HD).f251.webm (pass -k to keep)
[download] Downloading video 13 of 19
[youtube] ZDwSPiBaDco: Downloading webpage
[download] Destination: 20110218_Economics 421_521 - Econometrics - Winter 2011 - Lecture 13 (HD).f137.mp4
[download] 100% of 1.19GiB in 02:5668MiB/s ETA 00:008:41
[download] Destination: 20110218_Economics 421_521 - Econometric

[download] Destination: 20110311_Economics 421_521 - Econometrics - Winter 2011 - Lecture 19 (HD).f137.mp4
[download] 100% of 1.10GiB in 02:1922MiB/s ETA 00:006nown ETA
[download] Destination: 20110311_Economics 421_521 - Econometrics - Winter 2011 - Lecture 19 (HD).f251.webm
[download] 100% of 62.04MiB in 00:0579MiB/s ETA 00:006
[ffmpeg] Merging formats into "20110311_Economics 421_521 - Econometrics - Winter 2011 - Lecture 19 (HD).mkv"
Deleting original file 20110311_Economics 421_521 - Econometrics - Winter 2011 - Lecture 19 (HD).f137.mp4 (pass -k to keep)
Deleting original file 20110311_Economics 421_521 - Econometrics - Winter 2011 - Lecture 19 (HD).f251.webm (pass -k to keep)
[download] Finished downloading playlist: Economics 421 - Econometrics - Winter 2011


In [26]:
# Show avalible formats
with youtube_dl.YoutubeDL({'listformats':True,'format':'best[filesize<50M]'}) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=cSER5tjagqE'])

[youtube] cSER5tjagqE: Downloading webpage
[download] Destination: Goodie Bag 8 - Reactions (Intro to Solid-State Chemistry)-cSER5tjagqE.mp4
[download] 100% of 6.58MiB in 00:0085MiB/s ETA 00:007


In [15]:
for i,filelist in enumerate(filemultilist):
    with zipfile.ZipFile(f'showluo_{i}.zip', 'w') as zipMe:
        print('Part',i)
        for file in filelist:
            print('Adding',file)
            zipMe.write(file, compress_type=zipfile.ZIP_STORED) #zipfile.ZIP_DEFLATED

Adding 20140911_罗辑思维  23  击溃牛顿的钟表匠.mkv
Adding 20140911_罗辑思维  22  奸臣指南.mkv
Adding 20140911_罗辑思维 10  成名的代价.webm
Adding 20140911_罗辑思维  58  五问互联网(番外篇).mkv
Adding 20140911_罗辑思维  34  躺倒也能当英雄.mkv
Adding 20140911_罗辑思维 41  大公司：和蚂蚁一起舞蹈.mkv
Adding 20140911_罗辑思维  56 奇葩陪审团.mkv
Adding 20140911_罗辑思维  20 为泡妞之成功而读书.mkv
Adding 20140911_罗辑思维  26  一个被吃掉的少年.mkv
Adding 20140911_罗辑思维  36  大家都有拖延症.mkv
Adding 20140911_罗辑思维  18  腐败黑箱必被砸爆.mkv
Adding 20140911_罗辑思维  39  被排斥的中国人.mkv
Adding 20140911_罗辑思维  31 你的女神你不懂.mp4
Adding 20140911_罗辑思维  15  大国不能认死理.mkv
Adding 20140911_罗辑思维  43  土地私有,到底有多痛？.mkv
Adding 20140911_罗辑思维  16  杀死上帝的达尔文.mkv
Adding 20140911_罗辑思维 05  歪理邪说一“罗”筐.webm
Adding 20140911_罗辑思维 01 向死而生.webm
Adding 20140911_罗辑思维  33 官僚为什么当骗子.mkv
Adding 20140911_罗辑思维 07 夹缝中的80后.mkv
Adding 20140911_罗辑思维  38 一个公务员的海市蜃楼  Online Thinking Course.webm
Adding 20140911_罗辑思维  45  你怎么知道他该死.mkv
Adding 20140911_罗辑思维  21 岳飞为什么必须死.mp4
Adding 20140911_罗辑思维  37  政客林肯.mkv
Adding 20140911_罗辑思维  42  离开达尔文的日子.webm
Adding 20140911_罗辑思维 

folder_password = 'secret'

oc = owncloud.Client.from_public_link(public_link, password=folder_password)
oc.get_file('/sharedfile.zip', 'download/destination/sharedfile.zip')

# Selenium Driver Here

In [7]:
# Version 67
# From https://ungoogled-software.github.io/ungoogled-chromium-binaries/releases/linux_portable/64bit/
!wget -q https://github.com/Eloston/ungoogled-chromium-binaries/releases/download/67.0.3396.87-2/ungoogled-chromium_67.0.3396.87-2_linux.tar.xz
!tar -xf ungoogled-chromium_67.0.3396.87-2_linux.tar.xz

# From https://chromedriver.storage.googleapis.com/index.html
!wget -q https://chromedriver.storage.googleapis.com/2.41/chromedriver_linux64.zip
!unzip -o chromedriver_linux64.zip -d ./ungoogled-chromium_67.0.3396.87-1_linux

Archive:  chromedriver_linux64.zip
  inflating: ./ungoogled-chromium_67.0.3396.87-1_linux/chromedriver  


In [8]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import re
import os
from lxml import etree
import pandas as pd
import numpy as np

'''
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('log-level=3')
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_driver = os.getcwd() +"/ungoogled-chromium_67.0.3396.87-1_linux/chromedriver"
chrome_options.add_argument("--width=2560")
chrome_options.add_argument("--height=1440")
browser = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)
'''

'\nchrome_options = webdriver.ChromeOptions()\nchrome_options.add_argument(\'--headless\')\nchrome_options.add_argument(\'--disable-gpu\')\nchrome_options.add_argument(\'--incognito\')\nchrome_options.add_argument(\'log-level=3\')\nchrome_options.add_argument("--disable-dev-shm-usage")\nchrome_options.add_argument("--no-sandbox")\nchrome_driver = os.getcwd() +"/ungoogled-chromium_67.0.3396.87-1_linux/chromedriver"\nchrome_options.add_argument("--width=2560")\nchrome_options.add_argument("--height=1440")\nbrowser = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)\n'

In [9]:
class recommendation_panel(object):
    """An expectation for checking that the search tag has a minimum number in the document.
    locator - used to find the element
    min_recommendations - Minimum number of tags that should be found
    returns the WebElement once it has the minimum of tags
    """
    def __init__(self, locator, min_recommendations):
        self.locator = locator
        self.minrecmd = min_recommendations

    def __call__(self, driver):
        elements = driver.find_elements(*self.locator)   # Finding the referenced element
        if len(elements) >= self.minrecmd:
            return elements
        else:
            return False

class youtube(object):
    # init variables
    driver = None
    source = None
    element = None
    minrecmd = None
    vid = None
    meta = None
    
    def __init__(self,min_recommendations = 5):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--incognito')
        chrome_options.add_argument('log-level=3')
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--no-sandbox")
        chrome_driver = os.getcwd() +"/ungoogled-chromium_67.0.3396.87-1_linux/chromedriver"
        self.driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)
        self.minrecmd = min_recommendations
        
    def fetch(self, vid):
        # Ask browser to go to url
        #self.driver.execute_script("window.open('');")
        #self.driver.switch_to.window(self.driver.window_handles[1])
        self.vid = vid
        self.driver.set_window_position(0, 0)
        self.driver.set_window_size(1968, 1000)
        self.driver.get(f'https://www.youtube.com/watch?v={vid}')
        #self.driver.switch_to.window(driver.window_handles[1])
        # Wait until we have the recommendation panel
        self.element = WebDriverWait(self.driver, 10).until(
            recommendation_panel((By.CSS_SELECTOR, 'span#video-title'),self.minrecmd))
        self.source = self.driver.page_source
        meta = {}
        meta.update({y:x for x,y in zip([x.get_attribute('content') for x in self.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('name') in ['title','description','keywords']],[x.get_attribute('name') for x in self.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('name') in ['title','description','keywords']])})
        meta.update({y:x for x,y in zip([x.get_attribute('content') for x in self.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('itemprop') in ['duration','interactionCount','datePublished','genre']],[x.get_attribute('itemprop') for x in self.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('itemprop') in ['duration','interactionCount','datePublished','genre']])})
        meta.update({'thumbnail': x for x in [x.get_attribute('content') for x in self.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('property') in ['og:image']]})
        # {'property': 'og:image', 'content': 'https://i.ytimg.com/vi/5L31Wbrg7tE/maxresdefault.jpg'},})
        self.meta = meta
        
        #self.getscreenshot()
        #self.driver.close()
        #self.driver.switch_to.window(self.driver.window_handles[0])
        return self.element is not None
    
    def getsource(self):
        if self.source:
            return self.source
        else:
            return 'Empty buffer'#self.driver.page_source
    
    def getscreenshot(self):
        self.driver.save_screenshot('screenshot.png')
        return 'Done'
  
    def getrecommend(self):
        html = etree.HTML(self.source)
        vid = [str.split(x.attrib['href'],'=')[1] for x in html.findall(".//a[@id='thumbnail']") if ('href' in x.attrib.keys())]
        title = [x.attrib['title'] for x in html.findall(".//span[@id='video-title']")]
        channel = [x.text for x in html.findall(".//div[@id='metadata']/div/ytd-channel-name/div/div/yt-formatted-string")]
        views = [x.getchildren()[0].text.replace(' views','') if (len(x.getchildren()) > 1) else x.getchildren()[0].text for x in html.findall(".//div[@id='metadata-line']")]
        created = [x.getchildren()[1].text if (len(x.getchildren()) > 1) else 0 for x in html.findall(".//div[@id='metadata-line']")]
        result = pd.DataFrame(zip(vid,title,channel,views,created),columns=['ID','TITLE','CHANNEL','VIEWS','CREATED'])
        return result
    
    def transcript(self,vid=''):
        if vid == '':
            vid = self.vid
        resp = requests.get(f'https://hierogly.ph/api/transcribe?v={vid}',headers={'User-Agent':'Mozilla/5.0'})
        result = re.sub(r'\[[a-zA-Z0-9,.:\s]*\]','',resp.json()['text'])
        return result


In [13]:
yt = youtube(20)

In [14]:
yt.fetch('r6UDVAWtihA')

TimeoutException: Message: 


In [ ]:
from IPython.display import Image, HTML
yt.getscreenshot()
Image('screenshot.png')

In [ ]:
Image(yt.meta['thumbnail'])

In [ ]:
yt.getrecommend()

In [ ]:
yt.transcript()

In [24]:
# Get Video Properties
{y:x for x,y in zip([x.get_attribute('content') for x in yt.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('name') in ['title','description','keywords']],[x.get_attribute('name') for x in yt.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('name') in ['title','description','keywords']])}

{'title': 'Handling The Honeymoon Phase In Relationships | ZULA ChickChats | EP 100',
 'description': "It's our 100th episode of Zula Chickchats! And in this episode, we discuss couples' honeymoon phases and what happened after that.Thank you Bifesta for makin...",
 'keywords': 'honeymoon phase, singapore, relationship, pda, being comfortable around partner, boyfriend, girlfriend, bifesta, removing makeup, insecurities, bare face, revealing bare face'}

In [23]:
{y:x for x,y in zip([x.get_attribute('content') for x in yt.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('itemprop') in ['duration','interactionCount','datePublished','genre']],[x.get_attribute('itemprop') for x in yt.driver.find_elements(By.CSS_SELECTOR,"meta") if x.get_attribute('itemprop') in ['duration','interactionCount','datePublished','genre']])}

{'duration': 'PT11M44S',
 'interactionCount': '75185',
 'datePublished': '2020-08-21',
 'genre': 'People & Blogs'}

In [15]:
html = etree.HTML(yt.driver.page_source)

In [16]:
[x.attrib for x in html.findall(".//meta")]

[{'http-equiv': 'X-UA-Compatible', 'content': 'IE=edge'},
 {'name': 'theme-color', 'content': 'rgba(255,255,255,0.98)'},
 {'charset': 'UTF-8'},
 {'name': 'title', 'content': 'Handling The Honeymoon Phase In Relationships | ZULA ChickChats | EP 100'},
 {'name': 'description', 'content': "It's our 100th episode of Zula Chickchats! And in this episode, we discuss couples' honeymoon phases and what happened after that.Thank you Bifesta for makin..."},
 {'name': 'keywords', 'content': 'honeymoon phase, singapore, relationship, pda, being comfortable around partner, boyfriend, girlfriend, bifesta, removing makeup, insecurities, bare face, revealing bare face'},
 {'property': 'og:site_name', 'content': 'YouTube'},
 {'property': 'og:url', 'content': 'https://www.youtube.com/watch?v=5L31Wbrg7tE'},
 {'property': 'og:title', 'content': 'Handling The Honeymoon Phase In Relationships | ZULA ChickChats | EP 100'},
 {'property': 'og:image', 'content': 'https://i.ytimg.com/vi/5L31Wbrg7tE/maxresdefault

In [73]:
## Get the IDs from all the anchors
[str.split(x.attrib['href'],'=')[1] for x in html.findall(".//a[@id='thumbnail']") if ('href' in x.attrib.keys())]

['YSFvMJ7Sr9A',
 'nLOvDD9Bs1A&list',
 'xRk3XCYlSno',
 '-H9XpDHhLfM',
 'ikdbeBDAfzg',
 'xgAKV96pFbk',
 'SpSNmPKZOmk',
 'P8Ur0Yi38Sw',
 'i4Nn6Gx2Uv8',
 'xIEhVH7wQw4',
 'GMnwea3nfcg',
 '2zz8kP7Jf94',
 'tsIDyQPHLhs',
 'TZsJvY_FRMY',
 'O1Jkz9AXNTg',
 'YXJ8u16iEVI',
 '96J0qDggwOg',
 '25DOmcjbGAk',
 'o0jefVloo0o',
 'DdnSlF7x9oc']

In [74]:
# Recommended Video Channels
[x.text for x in html.findall(".//div[@id='metadata']/div/ytd-channel-name/div/div/yt-formatted-string")]

['Denpurna Nightcore',
 'YouTube',
 'Cool Nightcore',
 'Nightcore Cloud',
 'Shiro',
 'VvinChan Nightcore',
 'Kuna Unbrella',
 'Nightcore Disciple',
 'Legendary Nightcore',
 'Acoustic Songs Collection',
 'Trending TikTok',
 'Hazzard Overtake',
 'Sachi',
 'Acoustic Songs Collection',
 'JINIE',
 'Zein Nightcore',
 'Cool Nightcore',
 'Nightcore Cloud',
 'Anirose',
 "Xavi's Nightcore Lab"]

In [75]:
# Recommended Video IDs
[str.split(x.attrib['href'],'=')[1] for x in html.findall(".//span[@id='video-title']/../..")]

['YSFvMJ7Sr9A',
 'nLOvDD9Bs1A&list',
 'xRk3XCYlSno',
 '-H9XpDHhLfM',
 'ikdbeBDAfzg',
 'xgAKV96pFbk',
 'SpSNmPKZOmk',
 'P8Ur0Yi38Sw',
 'i4Nn6Gx2Uv8',
 'xIEhVH7wQw4',
 'GMnwea3nfcg',
 '2zz8kP7Jf94',
 'tsIDyQPHLhs',
 'TZsJvY_FRMY',
 'O1Jkz9AXNTg',
 'YXJ8u16iEVI',
 '96J0qDggwOg',
 '25DOmcjbGAk',
 'o0jefVloo0o',
 'DdnSlF7x9oc']

In [76]:
# Recommended Video Titles
[x.attrib['title'] for x in html.findall(".//span[@id='video-title']")]

['Best Nightcore Switching Vocals Mix 2018 ✪ 1 Hour Special ✪',
 'Mix - Nightcore Romantic MIX 2020 - Best Nightcore Switching Vocals Couple Romantic Song Mix',
 'New Nightcore Songs Mix 2020 - Top Nightcore Songs Hour Music 2020 - Best Nightcore Song Remix #6',
 'Best  Love Songs 2020 - Best Hits English  Cover Of Popular Songs /',
 '30 Minutes Nightcore Couple - Switching Vocals Mix (1k Subs Special)',
 'Best Nightcore 2020 - New OPM Nightcore Songs 2020 - New Tagalog Songs 2020 Playlist',
 '❤1Hour Romance Nightcore Mix ❤ #1',
 'Nightcore Opm love songs | New Opm Nightcore 2020',
 'Sad Nightcore 1 Hour Mix! (You must watch) Best Songs!',
 'Top English Acoustic Love Songs 2020 - Greatest Hits Ballad Acoustic Guitar Cover Of Popular Songs',
 'Tik Tok Songs 2020 * TikTok Music 2020 * TikTok Hits 2020 * Tik Tok Hot Trending 2020',
 '★ 30 minutes ♪ Nightcore Couple  Mix #3 💘',
 '30 Minutes Switching Vocals ↬ Nightcore  (Lyrics) (100 Subscribers Special)',
 'Best English Acoustic Love Song

In [80]:
# Get Popularity views
[x.getchildren()[0].text for x in html.findall(".//div[@id='metadata-line']")] # .replace(' views','')
# df['Val'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

['1M views',
 None,
 '358K views',
 '24K views',
 '816K views',
 '1.9K views',
 '303K views',
 '192K views',
 '2.5M views',
 '5M views',
 '336K views',
 '741K views',
 '668 views',
 '2.9M views',
 '137K views',
 '908K views',
 '94K views',
 '1.6M views',
 '368K views',
 '2.5K views']

In [42]:
pd.DataFrame(zip(vid,title,channel,views),columns=['ID','TITLE','CHANNEL','VIEWS'])['VIEWS'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

0      392000
1     8100000
2     1200000
3     1200000
4     3100000
5     4099999
6     8100000
7     3500000
8     2000000
9      335000
10      93000
11     898000
12    1700000
13     362000
14    3000000
15     548000
16    5400000
17    2900000
18    5200000
19      80000
Name: VIEWS, dtype: int64

In [91]:
# Get Created data
[x.getchildren()[1].text if (len(x.getchildren()) > 1) else 'None' for x in html.findall(".//div[@id='metadata-line']")]

['2 years ago',
 'None',
 '4 months ago',
 '3 months ago',
 '3 years ago',
 '2 weeks ago',
 '1 year ago',
 '2 months ago',
 '3 years ago',
 '1 month ago',
 '1 month ago',
 '2 years ago',
 '2 months ago',
 '1 month ago',
 '4 months ago',
 '3 months ago',
 '5 months ago',
 '1 year ago',
 '1 year ago',
 '2 years ago']

In [81]:
vid = [str.split(x.attrib['href'],'=')[1] for x in html.findall(".//a[@id='thumbnail']") if ('href' in x.attrib.keys())]
title = [x.attrib['title'] for x in html.findall(".//span[@id='video-title']")]
channel = [x.text for x in html.findall(".//div[@id='metadata']/div/ytd-channel-name/div/div/yt-formatted-string")]
views = [x.getchildren()[0].text for x in html.findall(".//div[@id='metadata-line']")]
created = [x.getchildren()[1].text for x in html.findall(".//div[@id='metadata-line']")]
pd.DataFrame(zip(vid,title,channel,views,created),columns=['ID','TITLE','CHANNEL','VIEWS','CREATED'])

IndexError: list index out of range

## BTO Transcripts

In [37]:
transcript('cYtSEj1v0ew')

"This is John and Jill. They are about to get married and start a family, but first they have to plan for their future and it's not buying the perfect home. Both John and Jill are working polygraph and the earn a combined monthly salary of $ 4,000. Ideally, they would like to live near their parents in the ground, so many options is easy to get confused now. Do you know what options you have, how welcome number option? 1, a 3-bedroom easy in some balance, a 3-bedroom easy and the balance will cost you about. Seven hundred ninety thousand dollars there's a monthly household income of four thousand dollars. You will be able to take a bank loan of up to two hundred and twenty thousand dollars. Next, let's start by it, you can receive a CPF housing grant releases of up to $ 30,000. This review is four hundred ninety thousand dollars, which has to be paid up front and and resupply suspend for thirty. Nine thousand five hundred dollars has to be paid cash, while the rest can be covered by CP

In [139]:
transcript('rb_adz0tkOw')

"Both of us, we plant a lot of strawberries la on each other's lands. What * screams * Hi, I'm Bolin, I'm Qing, I'm Peps Hi, I'm Leah, and this is Zula ChickChats. Okay, hi guys welcome to today's episode of ChickChats. Why Are you all very nervous? Yes, this is my first time on ChickChats Welcome And today's episode we'll be talking about being in the grey area as much requested from the comments section below. Ah So we hear you okay, we will do this episode for you guys. So my first question to you guys, then, is what happened. Okay, whoever wants can share first la What happened. Bolin, Let's start with Bolin. No Qing should start because she's the new person. Why me? Oh my god? Yes, What happened. I met the person through an app Okay, So we hit off like we talk, then after that move to Telegram. But the thing right we went out for so many times right. The places we go to right were like-, not the kind of place that you and your friends would go to. For example, For example, I need 

In [122]:
transcript('5L31Wbrg7tE')

"On one of the days um, we were supposed to have a date .... He came to my house like downstairs. Then I no time to draw eyebrows no time to draw anything right. Then we [ sat ] at the lamp post. There [, I made ] sure to choose the dimmest lamppost Hi. I ’ m Julian Hi. I ’ m Amanda Hi. I ’ m John Hi. I ’ m Leah and this is Zula ChickChats Hi guys welcome to today. ’ s, episode of Zula ChickChats And today we ’ re gon na be talking about- the honeymoon phase in relationships. We have 3 people over here to share uh- their honeymoon phases in their relationships. So…. The first Can you: why Can you explain what the honeymoon phase is I ...? What do you mean You don, ’ t know I ’ m married. I forgot about it. It ’ s been a long time ago. Okay, so the honeymoon phase, I guess, would be the period before your relationship starts getting serious, All the butterflies, all the romantic gestures, Oh ..., For the first question: maybe we ’ ll, let the audience know for you guys. How long have yo

In [140]:
transcript('cYtSEj1v0ew')

"This is John and Jill. They are about to get married and start a family, but first they have to plan for their future and it's not buying the perfect home. Both John and Jill are working polygraph and the earn a combined monthly salary of $ 4,000. Ideally, they would like to live near their parents in the ground, so many options is easy to get confused now. Do you know what options you have, how welcome number option? 1, a 3-bedroom easy in some balance, a 3-bedroom easy and the balance will cost you about. Seven hundred ninety thousand dollars there's a monthly household income of four thousand dollars. You will be able to take a bank loan of up to two hundred and twenty thousand dollars. Next, let's start by it, you can receive a CPF housing grant releases of up to $ 30,000. This review is four hundred ninety thousand dollars, which has to be paid up front and and resupply suspend for thirty. Nine thousand five hundred dollars has to be paid cash, while the rest can be covered by CP

In [145]:
transcript('mPHGGxZwICI')

"My name is I've been working for about two years. My name is Rob. I'Ve been working for about two years, six years, plus I think five years plus I'm very sure, let's see God my location off just nice kata-kata and they realize okay might as well. We just did I just graduated. Yes, I think two years ago yeah - and I guess it just seemed like a natural information from that February. For the column, one poet a key numbers, a thousand by hello, I'm Hansel - I just graduated from in yet last year, and now I'm doing an internship of jewelry come and your student, your tree, we have been together what six? Yes, I think with us always day. Video, what does the you need to be married before you can apply for the blueblack all controller? I mean you're gon na collect the key. Oh boss, you don't need to be married, like yeah, you can apply first, then you get true or false. It is possible to play pain, zero cash yeah. You can actually go share without paying any cash. It wasn't actually, when w